In [43]:
import requests
from bs4 import BeautifulSoup, Comment
from markdownify import markdownify as md
import re

In [84]:
# url = 'https://pmjay.gov.in/'
# url = 'https://pmjay.gov.in/whats-new?page=3'
# url = 'https://pmjay.gov.in/ayushmancapf/'
# url = 'https://pmjay.gov.in/ayushmancapf/ayushman-capf'
# url = 'https://pmjay.gov.in/ayushmancapf/resources'
# url = 'https://pmjay.gov.in/ayushmancapf/contact'
# url = 'https://pmjay.gov.in/contact-us'
# url = 'https://pmjay.gov.in/ayushman-mitra'
url = 'https://nha.gov.in/'

In [85]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [91]:
url = 'https://nha.gov.in/strapi/landing-page'
res = requests.get(url)
print(res.content)

b'{"id":1,"logo_link":null,"locale":"en","published_at":"2025-02-13T06:16:56.291Z","created_at":"2025-02-13T06:16:52.615Z","updated_at":"2026-02-19T10:38:57.144Z","landing_page_heading":[{"id":1,"heading_link_buttons":[{"id":4,"button_name":"Create Ayushman Card","button_link":"https://beneficiary.nha.gov.in/","heading":null,"link_icon":null},{"id":5,"button_name":"Create ABHA Number","button_link":"https://abha.abdm.gov.in/abha/v3/register","heading":null,"link_icon":null},{"id":6,"button_name":"Our YouTube Channel","button_link":"https://www.youtube.com/@AyushmnaNHA","heading":null,"link_icon":{"id":2,"name":"youtubeapp.svg","alternativeText":"","caption":"","width":24,"height":24,"formats":null,"hash":"youtubeapp_d79f7b88df","ext":".svg","mime":"image/svg+xml","size":11.73,"url":"/uploads/youtubeapp_d79f7b88df.svg","previewUrl":null,"provider":"local","provider_metadata":null,"created_at":"2025-02-13T06:55:48.972Z","updated_at":"2025-02-13T06:56:16.790Z"}}],"heading":{"id":2,"title"

In [90]:
res = requests.get('https://nha.gov.in/')
soup = BeautifulSoup(res.content, 'html.parser')
print(soup)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><link href="https://fonts.googleapis.com" rel="preconnect"/><link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/><link href="https://fonts.googleapis.com/css2?family=Roboto:ital,wght@0,100;0,300;0,400;0,500;0,700;1,100;1,300;1,400;1,500;1,700&amp;display=swap" rel="stylesheet"/><link href="https://fonts.googleapis.com/css?family=Open Sans" rel="stylesheet"/><meta content="width=device-width,initial-scale=1" name="viewport"/><script src="https://icdcdn.azureedge.net/embeddedct/icd11ect-1.3.js"></script><link href="https://fonts.cdnfonts.com/css/ds-digital" rel="stylesheet"/><link href="https://nha.gov.in/ip/ipfavicon.ico" rel="icon" type="image/png"><style>@import url('https://fonts.cdnfonts.com/css/ds-digital');</style><title>Integrated Portal</title><script defer="defer" src="/main.js"></script></link></head><body><div id="root"></div></body></html>


In [87]:
soup = BeautifulSoup(response.content, 'html.parser')
for tag in soup([
        "script", "style", "header", "footer",
        # "nav", "aside", "form", "iframe",
        # "noscript", "svg", "canvas", 
    ]):
        tag.decompose()

## Decomposing the skip to content text
for tag in soup.find_all("a"):
    text = tag.get_text(strip=True).lower()
    if text.startswith("skip"):
        tag.decompose()
        
# Decompose classes
classes = ['.topbar', '.footer_logos']
for class_tag in classes:
    for tag in soup.select(class_tag):
        tag.decompose()

## Removing html comments 
for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
    comment.extract()


# remove img locations
for img in soup.find_all('img'):
     img.decompose()

## Skipping hidden css content
for hidden in soup.find_all(style=True):
    if "display:none" in hidden["style"].replace(" ", "").lower():
        hidden.decompose()

for crumb in soup.find_all(class_=re.compile("breadcrumb", re.I)):
    crumb.decompose()

# Convert to Markdown
markdown_text = md(str(soup), heading_style="ATX")

# Remove excessive blank lines
markdown_text = re.sub(r"\n\s*\n\s*\n+", "\n\n", markdown_text)

# Remove empty headings
markdown_text = re.sub(r"^#{1,6}\s*$", "", markdown_text, flags=re.MULTILINE)

## remove spaces
markdown_text = re.sub(r"[ \t]+", " ", markdown_text)

# skip trailing spaces
markdown_text = "\n".join(line.rstrip() for line in markdown_text.splitlines())

with open('site_text.txt', 'w', encoding='utf-8') as f:
    f.write(markdown_text)

In [77]:
# for tag in soup.find_all(class_='views-field'):
#     print(tag.get_text().strip('\n'))

In [52]:
# for stuff in soup.find_all('div', class_ = 'gbb-row-wrapper'):
#     # print(stuff.get_text())
#     # with open('garbage.txt', 'w', encoding='utf-8') as f:
#     #     f.write(stuff.get_text())